# Assignment 2: Large Language Models for Text Classification

### CS 410/510 Large Language Models Fall 2024

#### Greg Witt

## Load Dataset

**Install Dependencies**

In [ ]:
!pip install datasets

In [19]:
from datasets import load_dataset
import random

# load the training set of tweets
ds_train = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split="train")
# the dataset labels {0 : negative, 1: neutral, 2: positive }
print(f"""tweet_sentiment_multilingual training dataset: 
    -----------------------------------------
        {ds_train} 
    """)
random_tweet_index = random.randint(0,1839)
print(f"""
    Random Tweet:
    {ds_train['text'][random_tweet_index]}

    Label: 
    {ds_train['label'][random_tweet_index] }
""")


tweet_sentiment_multilingual training dataset: 
    -----------------------------------------
        Dataset({
    features: ['text', 'label'],
    num_rows: 1839
}) 
    

    Random Tweet:
    okay i\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\u2019mon America aren\u2019t you sick of her yet? (sorry) 

    Label: 
    0



In [18]:

# load the training set of tweets
ds_validation = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split="validation")
# the dataset labels {0 : negative, 1: neutral, 2: positive }
print(f"""tweet_sentiment_multilingual validation set: 
    -----------------------------------------
        {ds_validation}
    """)
random_tweet_index = random.randint(0,324)
print(f"""
    Random Tweet:
    {ds_validation['text'][random_tweet_index]}

    Label: 
    {ds_validation['label'][random_tweet_index] }
""")


tweet_sentiment_multilingual validation set: 
    -----------------------------------------
        Dataset({
    features: ['text', 'label'],
    num_rows: 324
})
    

    Random Tweet:
    @user @user I think after Charlie Hebdo the French did NOT react as the US did after 9/11. But they may do this time around. 

    Label: 
    0



## Load the Models

**Load Dependencies**

In [ ]:
!pip install transformers

#### [Lama 3.2 1B](https://huggingface.co/meta-llama/Llama-3.2-1B) 

